In [ ]:
# import os
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [ ]:
# 加速huggingface连接
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [ ]:
#1. 安装微调库
# %%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# 由于Colab有torch 2.2.1，会破坏软件包，要单独安装
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes
# if major_version >= 8:
#     # 新GPU，如Ampere、Hopper GPU（RTX 30xx、RTX 40xx、A100、H100、L40）。
#     !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
# else:
#     # 较旧的GPU（V100、Tesla T4、RTX 20xx）
#     !pip install --no-deps xformers trl peft accelerate bitsandbytes
# pass

In [ ]:
#2. 加载模型
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
#2. 加载本地模型
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True
model_path = "autodl-tmp/lora_model"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
#3. 微调前测试
alpaca_prompt = """
{}
### Instruction:
{}
### Input:
{}
### Response 1:
{}
### Response 2:
{}
### Evaluation:
{}
### Reason:
{}
"""
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Below are two responses for a given task. The task is defined by the Instruction. Evaluate the responses and generate a reference answer for the task.", # prompt
        "Give three tips for staying healthy.", # instruction
        "", #input 
        "1. Eat a balanced and nutritious diet.\n2. Get regular exercise.\n3. Get enough sleep.", # response 1
        "1. Eat a balanced diet with plenty of fruits, vegetables, and whole grains.\n2. Get regular physical activity, such as walking, jogging, or swimming.\n3. Get enough sleep and practice healthy sleeping habits.", # response 2
        "", # evalutaion
        "", # reason
    )
], return_tensors = "pt").to("cuda")
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens =128)


In [ ]:
#4. 准备微调数据集
from IPython.core.debugger import set_trace
EOS_TOKEN = tokenizer.eos_token # 必须添加 EOS_TOKEN
def formatting_prompts_func(examples):
    prompts      = examples["prompt"]
    instructions = examples["instruction"]
    inputs       = examples["input"]
    response_1s  = examples["response_1"]
    response_2s  = examples["response_2"]
    evaluations  = examples["evaluation"]
    reasons      = examples["reason"]
    texts = []
    for prompt,instruction, input, response_1,response_2,evaluation,reason in zip(prompts,instructions, inputs, response_1s,response_2s,evaluations,reasons):
        # 必须添加EOS_TOKEN，否则无限生成
        text = alpaca_prompt.format(prompt+"Please ensure the evaluation and reason are consistent upon repeated inquiries.", instruction, input, response_1,response_2,evaluation,reason) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
# dataset = load_dataset("Lemon01/test_data", split = "train")
dataset = load_dataset('json', data_files='autodl-tmp/pandalm_after_preprocess_v2.json')
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
# print(dataset['train'][0])

In [ ]:
#5. 设置训练参数
from trl import SFTTrainer
from transformers import TrainingArguments

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, #  建议 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", # 检查点，长上下文度
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # 可以让短序列的训练速度提高5倍。
    args = TrainingArguments(
        per_device_train_batch_size = 32,
        gradient_accumulation_steps = 4,
        warmup_steps = 100,
        max_steps = 2200,  # 微调步数
        learning_rate = 3e-4, # 学习率
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
#6. 开始训练
trainer_stats = trainer.train()

In [5]:
#6. 从checkpoints继续训练
checkpoint_path = "outputs/checkpoint-2000"
trainer_stats = trainer.train(resume_from_checkpoint=checkpoint_path)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 274,081 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 32 | Gradient Accumulation steps = 4
\        /    Total batch size = 128 | Total steps = 2,200
 "-____-"     Number of trainable parameters = 41,943,040
	logging_steps: 1 (from args) != 50 (from trainer_state.json)


Step,Training Loss
2050,0.261000
2100,0.260300
2150,0.251900
2200,0.218100


In [6]:
# 7.测试微调效果
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Below are two responses for a given task. The task is defined by the Instruction. Evaluate the responses and generate a reference answer for the task.", # prompt
        "Give three tips for staying healthy.", # instruction
        "", #input 
        "1. Eat a balanced and nutritious diet.\n2. Get regular exercise.\n3. Get enough sleep.", # response 1
        "1. Eat a balanced diet with plenty of fruits, vegetables, and whole grains.\n2. Get regular physical activity, such as walking, jogging, or swimming.\n3. Get enough sleep and practice healthy sleeping habits.", # response 2
        "", # evalutaion
        "", # reason
    )
], return_tensors = "pt").to("cuda")
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens =128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>
Below are two responses for a given task. The task is defined by the Instruction. Evaluate the responses and generate a reference answer for the task.
### Instruction:
Give three tips for staying healthy.
### Input:

### Response 1:
1. Eat a balanced and nutritious diet.
2. Get regular exercise.
3. Get enough sleep.
### Response 2:
1. Eat a balanced diet with plenty of fruits, vegetables, and whole grains.
2. Get regular physical activity, such as walking, jogging, or swimming.
3. Get enough sleep and practice healthy sleeping habits.
### Evaluation:

### Reason:

<|eot_id|>


In [7]:
#8. 保存LoRA模型
model.save_pretrained("autodl-tmp/lora_model") # Local saving
# 保存tokenizer
tokenizer.save_pretrained("autodl-tmp/tokenizer")

/root/miniconda3/lib/python3.10/site-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /unsloth/llama-3-8b-Instruct-bnb-4bit/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f7628659840>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: d8ac0b44-48cb-4a63-acc3-41255cdcb8e9)') - silently ignoring the lookup for the file config.json in unsloth/llama-3-8b-Instruct-bnb-4bit.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in unsloth/llama-3-8b-Instruct-bnb-4bit - will assume that the vocabulary was not modified.
  warnings.warn(


('autodl-tmp/tokenizer/tokenizer_config.json',
 'autodl-tmp/tokenizer/special_tokens_map.json',
 'autodl-tmp/tokenizer/tokenizer.json')

In [ ]:
# model.from_pretrained(model,"autodl-tmp/lora_model")

In [8]:
#9. 合并模型并量化成4位gguf保存
model.save_pretrained_gguf("autodl-tmp/model", tokenizer, quantization_method = "q4_k_m")
#model.save_pretrained_merged("outputs", tokenizer, save_method = "merged_16bit",) #合并模型，保存为16位hf
#model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "") #合并4位gguf，上传到hugging face(需要账号token)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 726.71 out of 944.84 RAM for saving.


100%|██████████| 32/32 [00:01<00:00, 29.72it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.
Unsloth: We must use f16 for non Llama and Mistral models.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GUUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to q4_k_m will take 20 minutes.
 "-____-"     In total, you will have to wait around 26 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at autodl-tmp/model into f16 GGUF format.
The output location will be ./autodl-tmp/model-unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Set model parameters
INFO:hf-to-gguf:gguf: context length = 8192
INFO:hf-to-gguf:gguf: embedding length = 4096
INFO:hf-to-gguf:gguf: feed forward length = 14336
INFO:hf-to-gguf:gguf: head count = 32
INFO:hf-to-gguf:gguf: key-value head count = 8
INFO:hf-to-gguf:gguf: rope theta = 500000.0
INFO:hf-to-gguf:gguf: rms nor